# Starting point

We can start from the result of the previous version.

You don't need to run this cell, but you can use it for reference. The final code will be added at the end of the notebook.

```python
# Tower class
from typing import List

class Tower:
    def __init__(
        self,
        empty_level: str = "|",
        tower_base_fill: str = "=",
        disc_fill: str = "X",
    ) -> None:
        """Creates an empty tower"""
        self.empty_level = empty_level
        self.tower_base_fill = tower_base_fill
        self.disc_fill = disc_fill
        self._discs = []        

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

# HanoiTowers class
from typing import Tuple, Iterator

from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int, source: Tower, temporary: Tower, destination: Tower):
        self.num_discs = num_discs
        self.source: Tower = source
        self.destination: Tower = destination
        self.temporary: Tower = temporary
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3
    
    @staticmethod
    def default_towers(num_discs: int) -> "HanoiTowers":
        source = Tower()
        destination = Tower()
        temporary = Tower()
        
        for disc in range(num_discs, 0, -1):
            source.push(disc)
        
        return HanoiTowers(
            num_discs=num_discs,
            source=source,
            temporary=temporary,
            destination=destination,
        )

# HanoiPrinter class
from typing import List
from dataclasses import dataclass

class HanoiPrinter:
    def draw(self, towers: HanoiTowers) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(tower.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(tower.disc_fill * disc)
                else:
                    print(tower.empty_level)
            print(tower.tower_base_fill * towers.num_discs + "\n")


# HanoiSolver
class HanoiSolver:
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    def solve(self, towers: HanoiTowers):
        self._hanoi(
            source=towers.source,
            destination=towers.destination,
            temporary=towers.temporary,
            num_discs=towers.num_discs,
        )

# Hanoi class
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers.default_towers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)
```

# Dependency Inversion Principle

In this step, we will introduce abstractions for some of our classes, and try to illustrate a bit how that helps in terms of reducing dependencies in our code.

For brevity, we'll only be creating the interface for the ```Tower``` class, but it would definitely make sense to do the same for other classes, such as the ```HanoiPrinter``` and ```HanoiSolver``` classes.

## Abstracting the Tower class

First of all, we need to see what parts of the current ```Tower``` class should be in the *contract*. A way to think of it is: if we now re-implement a new Tower class, what methods HAVE to be there? In other words, what are the methods and attributes that other classes can rely on?

```python
from typing import List

class Tower:
    def __init__(
        self,
        empty_level: str = "|",
        tower_base_fill: str = "=",
        disc_fill: str = "X",
    ) -> None:
        """Creates an empty tower"""
        self.empty_level = empty_level
        self.tower_base_fill = tower_base_fill
        self.disc_fill = disc_fill
        self._discs = []        

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs
```

From these methods and attributes, we're using the different characters (to print), the ```push```/```pop``` methods (to manipulate the discs), and the ```__contains__``` (to perform operations like ```1 in tower```. However, we might decide to implement a new version of the ```Tower``` class that uses binary search to reduce the time to check if an element is contained in the tower, or that keeps the discs at specific indexes to provide constant lookup time. In any case, as long as we have the methods we mentioned above, we don't really care if the actual implementation is a list or not, or if the name of the attribute is ```_discs```, ```_items``` or something else.

So, it is a good practice to prefix anything that's not public from your class with an underscore. It will still be accessible from the outside, but you're indicating that it's part of the class' private "api", and as such other classes shouldn't rely on it.

As mentioned, there's are "interface"s in Python, but you can achieve the same result by using abstract classes, which are implemented in the [abc Python module](https://docs.python.org/3/library/abc.html). This is what we'll be using to implement "interfaces".

In [9]:
from abc import ABC, abstractmethod
from typing import Generic, TypeVar
T = TypeVar('T')


class Tower(ABC, Generic[T]):
    @property
    @abstractmethod
    def empty_level(self) -> str:
        pass
    
    @property
    @abstractmethod
    def tower_base_fill(self) -> str:
        pass
    
    @property
    @abstractmethod
    def disc_fill(self) -> str:
        pass
    
    @abstractmethod
    def push(self, disc: T) -> None:
        pass
    
    @abstractmethod
    def pop(self) -> T:
        pass
    
    @abstractmethod
    def __contains__(self, other: T) -> bool:
        pass

And this is how we would rewrite our "Tower" class (note that with the abstraction, we defined the different characters as properties)

In [10]:
from typing import List

class TowerWithList(AbstractTower[int]):
    def __init__(
        self,
        empty_level: str = "|",
        tower_base_fill: str = "=",
        disc_fill: str = "X",
    ) -> None:
        """Creates an empty tower"""
        self._empty_level = empty_level
        self._tower_base_fill = tower_base_fill
        self._disc_fill = disc_fill
        self._discs = []        

    @property
    def empty_level(self) -> str:
        return self._empty_level

    @property
    def tower_base_fill(self) -> str:
        return self._tower_base_fill
    
    @property
    def disc_fill(self) -> str:
        return self._disc_fill


    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)

    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

Before adapting the rest of classes, let's see if after this change everything still works as expected

In [8]:
h = Hanoi.create(num_discs=2)
h.draw_and_solve()

Initial State:

SOURCE Tower

|
X
XX
==

TEMPORARY Tower

|
|
|
==

DESTINATION Tower

|
|
|
==



Final State:

SOURCE Tower

|
X
XX
==

TEMPORARY Tower

|
|
|
==

DESTINATION Tower

|
X
XX
==



## Adapting the rest of classes

After this change, since we kept the name "Tower" for our interface, we can keep most of our code as it was. The exception for this is in the few places where we're instantiating towers, since you can't just instantiate an abstract class (you have an example of what happens if you try in the cell below).

After introducing this abstraction, you can see that the only point of hard coupling between e.g. the ```TowerWithList``` and the ```HanoiTowers``` class is the ```default_towers``` method, which hopefully makes clear why having these methods is not a very good practice in terms of maintainability of the code.

# Final Code

Here's the final code for the solution proposed:

In [2]:
# Tower interface
from abc import ABC, abstractmethod
from typing import Generic, TypeVar
T = TypeVar('T')


class AbstractTower(ABC, Generic[T]):
    @property
    @abstractmethod
    def empty_level(self) -> str:
        pass
    
    @property
    @abstractmethod
    def tower_base_fill(self) -> str:
        pass
    
    @property
    @abstractmethod
    def disc_fill(self) -> str:
        pass
    
    @abstractmethod
    def push(self, disc: T) -> None:
        pass
    
    @abstractmethod
    def pop(self) -> T:
        pass
    
    @abstractmethod
    def __contains__(self, other: T) -> bool:
        pass

# Concrete implementation of the Tower interface, using a list to store the discs
from typing import List

class TowerWithList(AbstractTower[int]):
    def __init__(
        self,
        empty_level: str = "|",
        tower_base_fill: str = "=",
        disc_fill: str = "X",
    ) -> None:
        """Creates an empty tower"""
        self._empty_level = empty_level
        self._tower_base_fill = tower_base_fill
        self._disc_fill = disc_fill
        self._discs = []        

    @property
    def empty_level(self) -> str:
        return self._empty_level

    @property
    def tower_base_fill(self) -> str:
        return self._tower_base_fill
    
    @property
    def disc_fill(self) -> str:
        return self._disc_fill


    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)

    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

# HanoiTowers class
from typing import Tuple, Iterator

from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int, source: Tower, temporary: Tower, destination: Tower):
        self.num_discs = num_discs
        self.source: Tower = source
        self.destination: Tower = destination
        self.temporary: Tower = temporary
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3
    
    @staticmethod
    def default_towers(num_discs: int) -> "HanoiTowers":
        source = TowerWithList()
        destination = TowerWithList()
        temporary = TowerWithList()
        
        for disc in range(num_discs, 0, -1):
            source.push(disc)
        
        return HanoiTowers(
            num_discs=num_discs,
            source=source,
            temporary=temporary,
            destination=destination,
        )

# HanoiPrinter class
from typing import List
from dataclasses import dataclass

class HanoiPrinter:
    def draw(self, towers: HanoiTowers) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(tower.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(tower.disc_fill * disc)
                else:
                    print(tower.empty_level)
            print(tower.tower_base_fill * towers.num_discs + "\n")


# HanoiSolver
class HanoiSolver:
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    def solve(self, towers: HanoiTowers):
        self._hanoi(
            source=towers.source,
            destination=towers.destination,
            temporary=towers.temporary,
            num_discs=towers.num_discs,
        )

# Hanoi class
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers.default_towers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)

As always, the proof is in the pudding:

In [3]:
h = Hanoi.create(num_discs=2)
h.draw_and_solve()

Initial State:

SOURCE Tower

|
X
XX
==

TEMPORARY Tower

|
|
|
==

DESTINATION Tower

|
|
|
==



Final State:

SOURCE Tower

|
X
XX
==

TEMPORARY Tower

|
|
|
==

DESTINATION Tower

|
X
XX
==



# Extra exercises / discussion

## Interface Segregation Principle

In this solution, we have abstracted the ```Tower``` class, and we defined the following properties / methods on it:

```python
class Tower(ABC, Generic[T]):
    def empty_level(self) -> str:
        #...
    def tower_base_fill(self) -> str:
        #...
    
    def disc_fill(self) -> str:
        # ...
    
    def push(self, disc: T) -> None:
        # ...
    
    def pop(self) -> T:
        # ...
    
    def __contains__(self, other: T) -> bool:
        # ...
```

How does this relate to the Interface Segregation Principle? Would you make any changes to this interface?

## Apply DIP to other classes

In theory, you could (and should) abstract any of the dependencies in the code, but if you just want a bit more practice, probably the ```HanoiPrinter```, ```HanoiSolver``` or ```HanoiTowers``` are good candidates to be abstracted.